<a href="https://www.kaggle.com/code/mohamedabi/notebook9f0151717d?scriptVersionId=160565224" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-sarcasm-detection/Mini Projet8.docx
/kaggle/input/arabic-sarcasm-detection/Sceintfic_Report_Model.pdf
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/LICENSE
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/README.md
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/training_data.csv
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/testing_data.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from scipy.sparse import hstack
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Load data
df = pd.read_csv('/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/training_data.csv')
print(df)
df_test = pd.read_csv('/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/testing_data.csv')
print(df_test)
# Preprocess data


                                                   tweet  sarcasm sentiment  \
0      "د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...    False       NEU   
1      "مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...    False       NEU   
2      “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...     True       NEG   
3      "@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...     True       NEG   
4      "قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...    False       NEU   
...                                                  ...      ...       ...   
12543  هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...     True       NEG   
12544  "لا اله الا الله💜#أيفون_البروفيسور https://t.c...    False       NEU   
12545  "RT @turkyepost: #أردوغان : إذا كان المرتكب غي...    False       NEU   
12546  RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...    False       POS   
12547        النت يلعن شكله طول الوقت تمام لين بدا الحفل     True       NEG   

      dialect  
0         msa  
1         msa  
2  

In [3]:

# Preprocess data
df['tweet'] = df['tweet'].str.strip('"').dropna()
df_test['tweet'] = df_test['tweet'].str.strip('"').dropna()

# Extract labels and texts for training set
labels, texts = df["sarcasm"], df[["tweet", "dialect"]]

# Extract labels and texts for test set
labels2, texts2 = df_test["sarcasm"], df_test[["tweet", "dialect"]]


In [4]:

# One-hot encode categorical columns ('dialect') for training set
encoder = OneHotEncoder(sparse_output=True)
categorical_encoded = encoder.fit_transform(df[['dialect']])
joblib.dump(encoder, 'encoder.joblib')  # Save the encoder


['encoder.joblib']

In [5]:

# TF-IDF Vectorization for training set
vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(df["tweet"])
joblib.dump(vectorizer, 'vectorizer.joblib')
X_encoded = hstack([categorical_encoded, vectorizer.transform(df["tweet"])])


In [6]:
# One-hot encode categorical columns ('dialect') for test set using the same encoder as training set
categorical_encoded_test = encoder.transform(df_test[['dialect']])
joblib.dump(encoder, 'encoder_test.joblib')  # Save the encoder



['encoder_test.joblib']

In [7]:
# TF-IDF Vectorization for test set using the same vectorizer as training set
X_encoded_test = hstack([categorical_encoded_test, vectorizer.transform(df_test["tweet"])])


In [8]:


# Define the Random Forest classifier with class weights
randomforest_classifier = RandomForestClassifier(class_weight='balanced', random_state=42)


In [9]:

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

grid_search = GridSearchCV(randomforest_classifier, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_encoded, labels)



GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='f1')

In [10]:
# Get the best model from the grid search
best_randomforest_classifier = grid_search.best_estimator_



In [11]:

# Evaluate the best model on the test set
y_pred_test = best_randomforest_classifier.predict(X_encoded_test)
classification_rep_test = classification_report(labels2, y_pred_test)

# Evaluate the best model on the training set
y_pred_train = best_randomforest_classifier.predict(X_encoded)
classification_rep_train = classification_report(labels, y_pred_train)

# Display the results for the training set
print("Best Model Classification Report (Training Set):\n", classification_rep_train)

# Display the results for the test set
print("Best Model Classification Report (Test Set):\n", classification_rep_test)

# Save the best model to a joblib file
joblib.dump(best_randomforest_classifier, 'best_sarcasm_model.joblib')

Best Model Classification Report (Training Set):
               precision    recall  f1-score   support

       False       0.94      0.81      0.87     10380
        True       0.45      0.74      0.56      2168

    accuracy                           0.80     12548
   macro avg       0.69      0.77      0.71     12548
weighted avg       0.85      0.80      0.82     12548

Best Model Classification Report (Test Set):
               precision    recall  f1-score   support

       False       0.79      0.73      0.76      2179
        True       0.41      0.49      0.44       821

    accuracy                           0.66      3000
   macro avg       0.60      0.61      0.60      3000
weighted avg       0.69      0.66      0.67      3000



['best_sarcasm_model.joblib']

In [12]:
accuracy = accuracy_score(labels2, y_pred_test)
print(accuracy)
accuracy2 = accuracy_score(labels, y_pred_train)
print(accuracy2)

0.6643333333333333
0.7988524067580491
